In [1]:
import numpy as np
import src.solutions.eigenmodes_part1 as eigen_part1
import src.visualizations as vis
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
from scipy.sparse import csr_matrix
import src.solutions.direct_diffusion as direct_diffusion
import src.solutions.leapfrog as leapfrog



In [2]:
N = 50

initial_square = eigen_part1.create_init_matrix_a(N)

initial_circle = eigen_part1.create_circle(N)
dependency_circle = eigen_part1.create_circle_dependency(N, initial_circle)

initial_rectangle = eigen_part1.create_init_matrix_a(N, rectangular=True)

eigenvalues_sq, eigenvectors_sq = eigsh(initial_square, k=3, which="SM")
eigenvalues_circ, eigenvectors_circ = eigsh(dependency_circle, k=3, which="SM")
eigenvalues_rect, eigenvectors_rect = eigsh(initial_rectangle, k=3, which="SM")

vis.visualize_different_shapes(eigenvectors_sq, eigenvalues_sq, eigenvectors_circ, eigenvalues_circ, eigenvectors_rect, eigenvalues_rect, N)


#### 3.1 D Computed eigenfrequencies as a function of size \( L \) for three different shapes: Square, Circle, and Rectangle. The eigenvalues are computed and only the negative eigenvalues are used to determine the eigenfrequencies as \( \omega = \sqrt{-\lambda} \). The results are plotted to analyze how the eigenfrequency spectrum changes with increasing \( L \).


In [ ]:
# Dictionaries to store eigenfrequencies
eigenfrequencies_sq = {}
eigenfrequencies_circ = {}
eigenfrequencies_rect = {}

sizes = [1, 2, 3, 4, 5] # L
N = 50

for L in sizes:

    # Square
    initial_sq = eigen_part1.create_init_matrix_a(L, N)
    eigenvalues_sq, _ = eigh(initial_sq)
    negative_eigenvalues_sq = eigenvalues_sq[eigenvalues_sq < 0] # only use negative eigenvalues for eigenfrequencies
    eigenfrequencies_sq[L] = np.sqrt(-negative_eigenvalues_sq) if len(negative_eigenvalues_sq) > 0 else []

    # Circle
    initial_circ = eigen_part1.create_circle(N)
    matrix_circ = eigen_part1.create_circle_dependency(L, N, initial_circ)
    sparse_matrix_circle = csr_matrix(matrix_circ)
    num_eigenvalues = sparse_matrix_circle.shape[0]
    eigenvalues_circ, _ = eigsh(sparse_matrix_circle, k=num_eigenvalues - 1, which="SM")
    negative_eigenvalues_circ = eigenvalues_circ[eigenvalues_circ < 0] # only use negative eigenvalues for eigenfrequencies
    eigenfrequencies_circ[L] = np.sqrt(-negative_eigenvalues_circ) if len(negative_eigenvalues_circ) > 0 else []

    # Rectangle
    initial_rect = eigen_part1.create_init_matrix_a(L, N, rectangular=True)
    eigenvalues_rect, _ = eigh(initial_rect)
    negative_eigenvalues_rect = eigenvalues_rect[eigenvalues_rect < 0] # only use negative eigenvalues for eigenfrequencies
    eigenfrequencies_rect[L] = np.sqrt(-negative_eigenvalues_rect) if len(negative_eigenvalues_rect) > 0 else []

# visualize plots
vis.eigenfrequencies_plot(sizes, eigenfrequencies_sq, eigenfrequencies_circ, eigenfrequencies_rect)

#### 3.1E Computed eigenmodes for \( N = 50 \), with parameters \( A = 1 \), \( B = 0 \), and \( c = 1 \). The first 3 eigenmodes were selected from the computed eigenvalues and eigenvectors. Using these eigenmodes, a time-dependent oscillation was applied. The eigenmodes were then visualized through an animated plot, showing how they evolve over time for the matrix. The animation is saved as a GIF.

In [ ]:
# Parameters
N = 50  # Matrix size
num_modes = 3  # Number of eigenmodes
t_values = np.linspace(0, 10, 100)  # Time steps
A = 1
B = 0
c = 1

# Compute eigenmodes
matrix_sq = eigen_part1.create_init_matrix_a(N) # for matrix shape
eigenvalues, eigenvectors = np.linalg.eigh(matrix_sq)
selected_eigenvalues, selected_eigenvectors = eigenvalues[:num_modes], eigenvectors[:N, :num_modes] # first 3 eigenmodes are selected

# Animation
vis.plot_eigenmodes(N, num_modes, selected_eigenvalues, selected_eigenvectors, t_values, A, B, c)

## Steady State Diffusion with a Direct solver
Uses a dependency matrix incorporating all the dependencies and a b that enforces the boundary conditions and source.  
This is implemented on a circular grid where the edges of the circle are fixed at 0. The grid is solved with different discretization steps. 

In [ ]:
#parameter values
nntjes = [200, 40, 20]
source_location = (0.6, 1.2)
diameter = 4

# create the grid and plot the converged grid 
converged_grids = direct_diffusion.direct_diffusion(nntjes, source_location, diameter)
vis.plot_diffusion_circle(converged_grids, nntjes)

## Leapfrog Method on Spring dynamics
using the leapfrog method to discretize spring dynamics, simulate for different spring constants (k)

#### Parameter Values

In [4]:
#parameters first experiment
ks = [16, 8, 4, 2, 1]
m=1
x = 1
v_0 = 0
deltat = 0.01

# extra parameters second experiment
freqs =  [1.4, 1.2, 1, 0.8]
k=1
time = 18
xs = np.linspace(-2, 2, 9, endpoint=True)

#### Simulation pure harmonic oscillator, visualizing position-velocity plot

In [ ]:
data_per_k = leapfrog.harmonic_oscillator_leapfrog(ks, deltat, x, v_0, m)

vis.vis_harmonic_oscillator(data_per_k)

#### Simulation Spring dynamics with extra time-dependent sinusoidal force, Phase plot with position vs velocity

In [ ]:
# saves all simulation steps for every frequency of extra force 
phases_for_freqs = dict()

# iterate over all frequencies for the time-dependent extra force
for freq in freqs:
    data_per_x0 = leapfrog.harmonic_oscillator_extra_force(k, deltat, xs, v_0, m, freq, time)
    phases_for_freqs[freq] = data_per_x0

# makes a phase plot for every frequency
vis.vis_phase_oscillator(phases_for_freqs, freqs)
